# 🧠 Fine-Tune TinyLlama with QLoRA
This notebook fine-tunes the `TinyLlama-1.1B-Chat-v1.0` model on CLI Q&A data using QLoRA with the PEFT library.
Ensure `full_qa_pairs.json` is uploaded to the Colab working directory before running.

In [ ]:
!pip install -q bitsandbytes==0.41.1
!pip install -q transformers accelerate peft datasets

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload full_qa_pairs.json

In [ ]:
import json
from datasets import Dataset

# Load dataset
with open('full_qa_pairs.json', 'r') as f:
    data = json.load(f)

# Format: 'Q: question\nA: answer'
formatted_data = [{'text': f"Q: {item['question']}\nA: {item['answer']}"} for item in data]
dataset = Dataset.from_list(formatted_data)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)

# Apply LoRA
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.1,
    bias='none',
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir='./lora-tinyllama',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy='epoch',
    report_to='none'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
# Train and save adapter
trainer.train()
model.save_pretrained('./lora-tinyllama')
tokenizer.save_pretrained('./lora-tinyllama')